In [11]:
from models import GnnerAT
from trainer import LightningWrapper
import pickle
import pytorch_lightning as pl

In [3]:
labels=['Method', 'Material', 'Task', 'Generic', 'OtherScientificTerm', 'Metric']
encoder='allenai/scibert_scivocab_uncased'
path = 'scierc_dataset.pk'

In [6]:
with open(path, 'rb') as f:
    data = pickle.load(f)
    dev_data = [i.values() for i in data['dev']]
    train_data = [i.values() for i in data['train']]

In [7]:
model = GnnerAT(labels=labels)

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
train_loader = model.create_dataloader(train_data, batch_size=4, num_workers=4, shuffle=False)
val_loader = model.create_dataloader(dev_data, batch_size=4, num_workers=4, shuffle=False)

In [10]:
lightning = LightningWrapper(model)

In [15]:
# logging
use_wandb = True

if use_wandb:
    from pytorch_lightning.loggers import WandbLogger
    logger = WandbLogger(project="GNNer")
else:
    logger = None

In [17]:
trainer = pl.Trainer(logger=logger, max_epochs=40, gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(lightning, train_loader, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params
----------------------------------
0 | model | GnnerAT | 111 M 
----------------------------------
111 M     Trainable params
0         Non-trainable params
111 M     Total params
445.592   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/urchade.zaratiana/.conda/envs/py37_UZA/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:229: UserWarning: You called `self.log('overlap', ...)` in your `validation_epoch_end` but the value needs to be floating point. Converting it to torch.float32.
  f"You called `self.log({self.meta.name!r}, ...)` in your `{self.meta.fx}` but the value needs to"
/home/urchade.zaratiana/.conda/envs/py37_UZA/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (44) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]